# 提取所有题目信息

In [7]:
import json
import pandas as pd

In [8]:
LANGUAGE_LIST = ["java"]
SUFFIX_MAP = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "c&cpp": "cpp",
    "java": "java"
}


def generate_python_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a python function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a python class , the class signature as below {code_signature}"


def generate_javascript_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a javascript function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a javascript class , the class signature as below {code_signature}"


def generate_typescript_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a typescript function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a typescript class , the class signature as below {code_signature}"


def generate_ccpp_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a cpp function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a cpp class , the class signature as below {code_signature}"


def generate_java_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a java function , the function signature as below {code_signature},package is org.real.temp, class is Answer"
    elif code_type == "class":
        return f"please write a java class , the function signature as below {code_signature},package is org.real.temp, class is Answer"


LANGUAGE_PROMPT_MAP = {
    "python": generate_python_prompt,
    "javascript": generate_javascript_prompt,
    "typescript": generate_typescript_prompt,
    "c&cpp": generate_ccpp_prompt,
    "java":generate_java_prompt
}

In [9]:
for language in LANGUAGE_LIST:
    # 读取excel表 获取数据信息
    excel_data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx", sheet_name=language)
    data = excel_data[excel_data['check'] == 'yes']
    # 遍历每一行
    question_array = []

    for index, row in data.iterrows():
        task_id = int(row["task_id"])
        code_type = row["code_type"]
        dir_path = f"../all/t{task_id}"
        if language == "java":
            signature_path = f"{dir_path}/{language}/Signature.{SUFFIX_MAP[language]}"
        else:
            signature_path = f"{dir_path}/{language}/signature.{SUFFIX_MAP[language]}"
        addition_info = str(row["addition_info"])
        if addition_info == 'nan':
            addition_info = ""
        with open(signature_path, "r", encoding="utf8") as signature_file:
            code_signature = signature_file.read()
        if language == "java":
            test_path = f"{dir_path}/{language}/Tester.{SUFFIX_MAP[language]}"
        else:
            test_path = f"{dir_path}/{language}/test.{SUFFIX_MAP[language]}"
        with open(test_path, "r", encoding="utf8") as test_file:
            code_test = test_file.read()
        prompt = LANGUAGE_PROMPT_MAP[language](code_signature, code_type)
        question_info = {
            "task_id": task_id,
            "code_type": code_type,
            "code_signature": code_signature,
            "code_language": language,
            "test_code": code_test,
            "prompt": prompt,
            "addition_info": addition_info

        }
        question_array.append(question_info)
        print(f"task_id:{task_id} signature:{len(code_signature)} test:{len(code_test)}")
    with open(f"./question/{language}.json", "w", encoding="utf8") as question_file:
        json_str = json.dumps(question_array)
        question_file.write(json_str)
        question_file.flush()
    with open(rf"E:\code\code_back\python_project\llm\qa\question\{language}.json", "w",
              encoding="utf8") as question_file:
        json_str = json.dumps(question_array)
        question_file.write(json_str)
        question_file.flush()

task_id:170 signature:1316 test:1378
task_id:173 signature:824 test:1944
task_id:174 signature:1134 test:1341
task_id:175 signature:416 test:3963
task_id:176 signature:855 test:3293
task_id:177 signature:480 test:2544
task_id:178 signature:268 test:1365
task_id:179 signature:447 test:4488
task_id:180 signature:374 test:1934
task_id:181 signature:369 test:3074
task_id:182 signature:528 test:4152
task_id:348 signature:529 test:1121
task_id:349 signature:781 test:2287
task_id:350 signature:391 test:1312
task_id:351 signature:682 test:1008
task_id:352 signature:603 test:1164
task_id:608 signature:236 test:1072
task_id:609 signature:345 test:1665
task_id:610 signature:205 test:1914
task_id:611 signature:200 test:1813
task_id:612 signature:414 test:2658
task_id:614 signature:284 test:1901
task_id:615 signature:488 test:1670
task_id:616 signature:299 test:1561
task_id:617 signature:986 test:1802
task_id:619 signature:333 test:2298
task_id:620 signature:298 test:2002
task_id:622 signature:397 